In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image
import torchvision.models as models

import numpy as np
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import pickle
import time
import pandas as pd
from google.colab import files

from google.colab import drive
drive.mount('/content/gdrive' )
%cd '/content/gdrive/MyDrive/mini-proj3_data/'

In [ ]:
# //////////////////////////////////////////////////////////////////////////////
# ////////////////////////// Functions and Classes /////////////////////////////
# //////////////////////////////////////////////////////////////////////////////

class MyDataset(Dataset):
    def __init__(self, img_file, label_file, transform=None, idx = None):
        self.data = pickle.load( open( img_file, 'rb' ), encoding='bytes')
        self.targets = np.genfromtxt(label_file, delimiter=',', skip_header=1)[:,1:]

        if idx is not None:
          self.targets = self.targets[idx]
          self.data = self.data[idx]
        self.transform = transform

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        img, target = self.data[index], int(self.targets[index])
        img = Image.fromarray(img.astype('uint8'), mode='L')

        if self.transform is not None:
           img = self.transform(img)

        return img, target



class Net1(nn.Module):
    # This part defines the layers
    def __init__(self):
        super(Net1, self).__init__()

        self.conv1 = nn.Conv2d(1, 3, kernel_size=5)
        self.conv2 = nn.Conv2d(3, 6, kernel_size=5)

        self.fc1 = nn.Linear(2262, 9)

    def forward(self, x):
        
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))

        # This layer is an imaginary one. It simply states that we should see each member of x
        # as a vector of 320 elements, instead of a tensor of 20x4x4 (Notice that 20*4*4=320)
        x = x.view(-1, 2262)

        # Feedforeward layers. Remember that fc1 is a layer that goes from 320 to 50 neurons
        x = (self.fc1(x))

        # We should put an appropriate activation for the output layer.
        return F.log_softmax(x)


class Net2(nn.Module):
    # This part defines the layers
    def __init__(self):
        super(Net2, self).__init__()

        self.conv1 = nn.Conv2d(1, 3, kernel_size=5)
        self.conv2 = nn.Conv2d(3, 6, kernel_size=5)

        self.fc1 = nn.Linear(2262, 9)

    def forward(self, x):
        
        x = F.tanh(F.max_pool2d(self.conv1(x), 2))
        x = F.tanh(F.max_pool2d(self.conv2(x), 2))

        # This layer is an imaginary one. It simply states that we should see each member of x
        # as a vector of 320 elements, instead of a tensor of 20x4x4 (Notice that 20*4*4=320)
        x = x.view(-1, 2262)

        # Feedforeward layers. Remember that fc1 is a layer that goes from 320 to 50 neurons
        x = (self.fc1(x))

        # We should put an appropriate activation for the output layer.
        return F.log_softmax(x)


class Net3(nn.Module):
    # This part defines the layers
    def __init__(self):
        super(Net3, self).__init__()

        self.conv1 = nn.Conv2d(1, 6, kernel_size=9)
        self.conv2 = nn.Conv2d(6, 10, kernel_size=7)
        self.conv3 = nn.Conv2d(10, 20, kernel_size=5)

        self.fc1 = nn.Linear(3220, 1000)
        self.fc2 = nn.Linear(1000, 300)
        self.fc3 = nn.Linear(300, 9)

    def forward(self, x):
        
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = self.conv3(x)

        # This layer is an imaginary one. It simply states that we should see each member of x
        # as a vector of 320 elements, instead of a tensor of 20x4x4 (Notice that 20*4*4=320)
        x = x.view(-1, 3220)

        # Feedforeward layers. Remember that fc1 is a layer that goes from 320 to 50 neurons
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        # Output layer
        x = self.fc3(x)

        # We should put an appropriate activation for the output layer.
        return F.log_softmax(x)



class Net4(nn.Module):
    # This part defines the layers
    def __init__(self):
        super(Net4, self).__init__()

        self.conv1 = nn.Conv2d(1, 6, kernel_size=9)
        self.conv2 = nn.Conv2d(6, 10, kernel_size=7)
        self.conv3 = nn.Conv2d(10, 20, kernel_size=5)

        self.fc1 = nn.Linear(3220, 1000)
        self.fc2 = nn.Linear(1000, 300)
        self.fc3 = nn.Linear(300, 9)

    def forward(self, x):
        
        x = F.tanh(F.max_pool2d(self.conv1(x), 2))
        x = F.tanh(F.max_pool2d(self.conv2(x), 2))
        x = self.conv3(x)

        # This layer is an imaginary one. It simply states that we should see each member of x
        # as a vector of 320 elements, instead of a tensor of 20x4x4 (Notice that 20*4*4=320)
        x = x.view(-1, 3220)

        # Feedforeward layers. Remember that fc1 is a layer that goes from 320 to 50 neurons
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))

        # Output layer
        x = self.fc3(x)

        # We should put an appropriate activation for the output layer.
        return F.log_softmax(x)

        

def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data = data.to(device)
    target = target.to(device)
    optimizer.zero_grad()
    output = F.log_softmax(network(data))
    loss = F.nll_loss(output, target) #negative log liklhood loss
    loss.backward()
    optimizer.step()
    if batch_idx % 20 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*batchsize) + ((epoch-1)*len(train_loader.dataset)))
      torch.save({
                  'Lepoch': epoch,
                  'network_state_dict':network.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'Ltotal_time': total_time,
                  'Ltrain_losses': train_losses,
                  'Ltrain_counter':train_counter,
                  'Ltest_losses':test_losses,
                  'Lloss': loss}, './trainsaves/model.pth')
      # torch.save(optimizer.state_dict(), './trainsaves/optimizer.pth')


def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    test_loader = valid_loader
    for data, target in test_loader:
      data = data.to(device)
      target = target.to(device)
      output = F.log_softmax(network(data))
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nValid set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [ ]:
# //////////////////////////////////////////////////////////////////////////////
# /////////////////////// Inputs and Initialization ////////////////////////////
# //////////////////////////////////////////////////////////////////////////////

batchsize = 64       # Specify the batch size
if_validation = True  # Specify if you want to consider a part of training data as validation set
if_shuffle = True     # Specify if you want to shuffle the training data before training process

learningrate = 0.01
moment = 0.9
num_of_epochs = 50
random_seed = 0

if if_validation:
  train_idx = range(0,50000)
  valid_idx = range(50000,60000)
else:
  train_idx = None
# //////////////////////////////////////////////////////////////////////////////

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])


train_dataset = MyDataset('./Train.pkl', './TrainLabels.csv', transform=img_transform, idx=train_idx)
train_loader = DataLoader(train_dataset, batch_size=batchsize, shuffle=if_shuffle)

if if_validation:
  valid_dataset = MyDataset('./Train.pkl', './TrainLabels.csv', transform=img_transform, idx=valid_idx)
  valid_loader = DataLoader(valid_dataset, batch_size=batchsize, shuffle=if_shuffle)
else:
  valid_loader = train_loader


In [ ]:
# //////////////////////////////////////////////////////////////////////////////
# ///////////////////////////// Learning Stage /////////////////////////////////
# //////////////////////////////////////////////////////////////////////////////

network = Net1().to(device)
# network = models.resnet152().to(device)
# network.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False).to(device)
# network.fc = nn.Linear(in_features=2048, out_features=9, bias=True).to(device)


# optimizer = optim.SGD(network.parameters(), lr=learningrate, momentum=moment)
optimizer = optim.Adam(network.parameters())

total_time = []
train_losses = []
train_counter = []
test_losses = []

for epoch in range(1, num_of_epochs+1):
  start_time = time.clock()
  train(epoch)
  run_time = (time.clock() - start_time)
  total_time.append(run_time)

  test()

mean_time = sum(total_time) / len(total_time)
print('The mean run time for each epoch was:', mean_time, 'seconds')

In [ ]:
# //////////////////////////////////////////////////////////////////////////////
# ////////////////////////// Continue saved work ///////////////////////////////
# //////////////////////////////////////////////////////////////////////////////


network = Net1().to(device)
# network = models.resnet152().to(device)
# network.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False).to(device)
# network.fc = nn.Linear(in_features=2048, out_features=9, bias=True).to(device)

# optimizer = optim.SGD(network.parameters(), lr=learningrate, momentum=moment)
optimizer = optim.Adam(network.parameters())

# //////////////////////////////////////////////////////////////////////////////

checkpoint = torch.load('./trainsaves/model.pth')

network.load_state_dict(checkpoint['network_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
lastepoch = checkpoint['Lepoch']
loss = checkpoint['Lloss']
total_time = checkpoint['Ltotal_time']
train_losses = checkpoint['Ltrain_losses']
train_counter = checkpoint['Ltrain_counter']
test_losses = checkpoint['Ltest_losses']

# //////////////////////////////////////////////////////////////////////////////

for epoch in range(lastepoch+1, lastepoch+1+2):
  start_time = time.clock()
  train(epoch)
  run_time = (time.clock() - start_time)
  total_time.append(run_time)

  test()

mean_time = sum(total_time) / len(total_time)
print('The mean run time for each epoch was:', mean_time, 'seconds')


In [ ]:
# //////////////////////////////////////////////////////////////////////////////
# /////// Prediction of test data and download the labels in CSV format ////////
# //////////////////////////////////////////////////////////////////////////////

# Here, the values of "TestdummyLabels.csv" is not used. Just put a random file with 10000 labels (it is important that the size be 10000)
testdataset = MyDataset('./Test.pkl', './TestdummyLabels.csv', transform=img_transform, idx=None)
testloader = DataLoader(testdataset, shuffle=False)

j=1
network.eval()
y_guess = []
with torch.no_grad():
  for data, target in testloader:
        data = data.to(device)
        outputarray = F.log_softmax(network(data))
        outputlabel = outputarray.data.max(1, keepdim=True)[1]
        if j % 500 == 0:
          # print(type(outputlabel.item()), type(outputlabel))
          print('The prediction is done up to label index:', j)
        j = j+1
        y_guess.append(outputlabel.item())

y_guess2 = [x+5 for x in y_guess] # This line adds the previously subtracted 5 to the labels
y_guess_df = pd.DataFrame(y_guess2, columns=['class'])
y_guess_df.to_csv('KaggleLabels.csv', columns=['class'])
files.download("KaggleLabels.csv")